Resume Score

In [ ]:
!pip install transformers


Load Data

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Job_Resume_Dataset.xlsx to Job_Resume_Dataset.xlsx


In [ ]:
import pandas as pd
import io

# Assuming the file is named 'Job_Resume_Dataset.xlsx'
df = pd.read_excel(io.BytesIO(uploaded['Job_Resume_Dataset.xlsx']))
print(df.head())


                                     Job Description  \
0  Seeking a software engineer with experience in...   
1  Looking for a machine learning engineer profic...   
2  Hiring a full-stack developer familiar with Ja...   
3  Require an AI specialist with expertise in com...   
4  Data scientist needed, proficient in statistic...   

                                         Resume Text  Relevance Score  
0  Software developer with 5 years of experience ...               90  
1  Experienced machine learning engineer with pro...               95  
2  Full-stack developer knowledgeable in JavaScri...               85  
3  AI specialist with a PhD focused on computer v...               95  
4  Data scientist with 8 years in the field, expe...               90  


Prepare Data for Training

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Example of tokenizing the first entry
job_desc = df['Job Description'][0]
resume_text = df['Resume Text'][0]
inputs = tokenizer(job_desc, resume_text, return_tensors='pt', padding=True, truncation=True, max_length=512)
print(inputs)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

{'input_ids': tensor([[  101,  6224,  1037,  4007,  3992,  2007,  3325,  1999, 18750,  1998,
          2951,  5090,  1012,   102,  4007,  9722,  2007,  1019,  2086,  1997,
          3325,  1999, 18750,  1010,  9262,  1010,  1998,  2951,  5090,  1012,
           102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]])}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)  # We use 1 label for regression


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    eval_strategy="epoch"
)

In [ ]:
from sklearn.model_selection import train_test_split

train_df, eval_df = train_test_split(df, test_size=0.1)


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_data(df):
    return tokenizer(df['Job Description'].tolist(), df['Resume Text'].tolist(), padding="max_length", truncation=True, max_length=512, return_tensors="pt")

train_encodings = tokenize_data(train_df)
eval_encodings = tokenize_data(eval_df)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
import torch

class JobResumeDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = JobResumeDataset(train_encodings, train_df['Relevance Score'].tolist())
eval_dataset = JobResumeDataset(eval_encodings, eval_df['Relevance Score'].tolist())


In [ ]:
from transformers import Trainer, TrainingArguments

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# Train the model
trainer.train()


<ipython-input-11-91e3e1f77e1a>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss
1,No log,7241.182129
2,No log,7236.716309
3,No log,7230.517090


<ipython-input-11-91e3e1f77e1a>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


TrainOutput(global_step=18, training_loss=6813.559895833333, metrics={'train_runtime': 828.4058, 'train_samples_per_second': 0.163, 'train_steps_per_second': 0.022, 'total_flos': 35519673553920.0, 'train_loss': 6813.559895833333, 'epoch': 3.0})

In [ ]:
# Save the model and the tokenizer
model_path = "/content/drive/MyDrive/fine_tuned_bert"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)


('/content/drive/MyDrive/fine_tuned_bert/tokenizer_config.json',
 '/content/drive/MyDrive/fine_tuned_bert/special_tokens_map.json',
 '/content/drive/MyDrive/fine_tuned_bert/vocab.txt',
 '/content/drive/MyDrive/fine_tuned_bert/added_tokens.json')

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer

model_path = "/content/drive/MyDrive/fine_tuned_bert"
from transformers import AutoTokenizer, AutoModel


tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path)


In [ ]:
!pip install PyMuPDF


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 22.4 MB/s eta 0:00:00


In [ ]:
!pip install PyMuPDF
import fitz # Import the fitz module from PyMuPDF

def get_input():
    print("Please choose an input method:\n1. Upload file (PDF or TXT)\n2. Enter text manually")
    choice = input("Enter 1 or 2: ")
    if choice == '1':
        uploaded = files.upload()
        for name, content in uploaded.items():
            if name.lower().endswith('.pdf'):
                doc = fitz.open(stream=content, filetype='pdf') # Now you can use fitz.open
                text = ''
                for page in doc:
                    text += page.get_text()
                return text
            elif name.lower().endswith('.txt'):
                return content.decode('utf-8')
            else:
                print(f"Unsupported file type: {name}")
                return None
    elif choice == '2':
        return input("Enter your text here: ")
    else:
        print("Invalid input. Please enter 1 or 2.")
        return get_input()  # Recursive call to handle incorrect entries

print("Upload or Enter Job Description:")
job_description = get_input()
print("Upload or Enter Candidate Resume:")
candidate_resume = get_input()

Upload or Enter Job Description:
Please choose an input method:
1. Upload file (PDF or TXT)
2. Enter text manually
Enter 1 or 2: 1


Saving Sample Job Description.pdf to Sample Job Description.pdf
Upload or Enter Candidate Resume:
Please choose an input method:
1. Upload file (PDF or TXT)
2. Enter text manually
Enter 1 or 2: 1


Saving Sample CV.docx.pdf to Sample CV.docx.pdf


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

def get_embedding(text):
    encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        model_output = model(**encoded_input)
    if 'pooler_output' in model_output:
        embedding = model_output.pooler_output.squeeze()
    else:
        embedding = model_output.last_hidden_state.mean(dim=1).squeeze()
    return embedding.numpy()



In [ ]:
import re

In [ ]:
def extract_skills(text):
    skill_pattern = re.compile(r'(\d+)\s+years?\s+of\s+([\w+]+)', re.IGNORECASE)
    skills = skill_pattern.findall(text)
    return {skill.lower(): int(years) for years, skill in skills}

def extract_experience(text):
    # Looks for the number of years of experience mentioned in the text
    experience_pattern = re.compile(r'(\d+)\s+years?\s+of\s+experience', re.IGNORECASE)
    experience_matches = experience_pattern.findall(text)
    if experience_matches:
        return int(experience_matches[0])  # Assuming the first match is the overall experience required
    return 0  # Default if no experience is found

def extract_position(text):
    # Looks for common job positions like 'Senior Developer', 'Junior Developer', etc.
    position_pattern = re.compile(r'(senior|junior)\s+(developer|engineer|manager)', re.IGNORECASE)
    position_match = position_pattern.search(text)
    return position_match.group(0) if position_match else "Unknown"

In [ ]:
def calculate_skill_match(desc_skills, cv_skills):
    skill_match = {}
    total_score = 0
    for skill, years_required in desc_skills.items():
        years_have = cv_skills.get(skill, 0)
        match_percentage = (years_have / years_required) * 100 if years_required > 0 else 0
        skill_match[skill] = match_percentage
        total_score += match_percentage
    average_skill_match = total_score / len(desc_skills) if desc_skills else 0
    return min(average_skill_match, 100)  # Cap the score at 100%

def calculate_experience_match(job_experience, resume_experience):
    match_score = min((resume_experience / job_experience) * 100, 100) if job_experience else 0
    return min(match_score, 100)  # Cap the score at 100%

def calculate_position_match(job_position, resume_position):
    match_score = 100 if job_position.lower() == resume_position.lower() else 50
    return min(match_score, 100)  # Cap the score at 100%

In [ ]:
def calculate_matching_score(job_description, resume, skill_weight=0.5, experience_weight=0.3, position_weight=0.2):
    # Get BERT similarity score
    embedding1 = get_embedding(job_description)
    embedding2 = get_embedding(resume)
    text_similarity = cosine_similarity([embedding1], [embedding2])[0][0] * 100

    # Extract skills from both job description and resume
    desc_skills = extract_skills(job_description)
    cv_skills = extract_skills(resume)

    # Extract experience and position from job description and resume
    job_experience = extract_experience(job_description)
    resume_experience = extract_experience(resume)

    job_position = extract_position(job_description)
    resume_position = extract_position(resume)

    # Calculate skill, experience, and position matching
    skill_match_score = calculate_skill_match(desc_skills, cv_skills)
    experience_match_score = calculate_experience_match(job_experience, resume_experience)
    position_match_score = calculate_position_match(job_position, resume_position)

    # Cap scores at 100% for each factor
    text_similarity = min(text_similarity, 100)  # Cap text similarity at 100%

    # Calculate final matching score based on weighted contributions
    mScore = (text_similarity * skill_weight) + (skill_match_score * skill_weight) + (experience_match_score * experience_weight) + (position_match_score * position_weight)

    return min(mScore, 100)  # Ensure final score is capped at 100%

In [ ]:
# Example usage
#job_description = "The candidate should have 10 years of Java experience, 7 years of Python, and 3 years of C++. The position is for a Senior Developer with at least 5 years of experience."
#candidate_resume = "I have 4 years of Java experience, 1 years of Python, and 1 years of C++. I am currently a Junior Developer with 1 years of experience."
final_matching_score = calculate_matching_score(job_description, candidate_resume)
print(f"Final Matching Score: {final_matching_score:.2f}%")

Final Matching Score: 68.19%
